<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/llmu/co_aws_ch4_semantic_search.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

# Semantic Search Using Cohere Embed on Amazon Bedrock

Text embeddings are numerical representations created by language models that convert text into vectors. They capture and encode the context of a document. These vectors store a wealth of context about the documents they represent, opening up the possibility of a variety of applications, from semantic search and retrieval-augmented generation (RAG) to topic modeling and text classification.

Cohere's Embed model, available on Amazon Bedrock, is a powerful text embeddings model that offers these capabilities. This model supports over 100 languages and is unique among text embedding models due to its emphasis on document quality for applications like semantic search.

In this notebook, we'll explore how to use Cohere's Embed model on Amazon Bedrock. 

# Setup

First, let's install and import the necessary libraries and set up our Cohere client using the cohere SDK. To use Bedrock, we create a BedrockClient by passing the necessary AWS credentials.

In [ ]:
! pip install cohere pandas hnswlib -q

In [3]:
import pandas as pd
import cohere
import hnswlib
import re

In [ ]:
import cohere

# Create Bedrock client via the native Cohere SDK
# Contact your AWS administrator for the credentials
co = cohere.BedrockClient(
    aws_region="YOUR_AWS_REGION",
    aws_access_key="YOUR_AWS_ACCESS_KEY_ID",
    aws_secret_key="YOUR_AWS_SECRET_ACCESS_KEY",
    aws_session_token="YOUR_AWS_SESSION_TOKEN",
)

# Download dataset

We use a dataset (MultiFIN) containing a list of real-world article headlines covering 15 languages (English, Turkish, Danish, Spanish, Polish, Greek, Finnish, Hebrew, Japanese, Hungarian, Norwegian, Russian, Italian, Icelandic, and Swedish). This is an open-source dataset curated for financial natural language processing (NLP) and is available on a [GitHub repository](https://github.com/RasmusKaer/MultiFin).

In our case, we’ve created a CSV file with MultiFIN’s data, as well as a column with translations. We don’t use this column to feed the model; we use it to help us follow along when we print the results for those who don’t speak Danish or Spanish. We point to that CSV to create our dataframe.

In [5]:
url = "https://raw.githubusercontent.com/cohere-ai/cohere-aws/main/notebooks/bedrock/multiFIN_train.csv"
df = pd.read_csv(url)

# Inspect dataset
df.head(5)

,text,labels,lang,id,translation
0,Revenue Recognition,['Accounting & Assurance'],English,Israel-4145,Revenue Recognition
1,Más de la mitad de las empresas españolas fuer...,['Financial Crime'],Spanish,Spain-2044,More than half of the Spanish companies were v...
2,Wynagrodzenie netto w Polsce to średnio 71% pe...,['Human Resource'],Polish,Poland-1567,The net salary in Poland is an average of 71% ...
3,Time to talk: What has to change for women at ...,['Human Resource'],English,Turkey-5447,Time to talk: What has to change for women at ...
4,Total Retail 2017,['Retail & Consumers'],English,Spain-1981,Total Retail 2017


# Pre-Process Dataset

MultiFIN has over 6,000 records in 15 different languages. For our example use case, we focus on three languages: English, Spanish, and Danish.
For this, we’ll need to do some pre-processing steps. First we remove the duplicates, remove the languages other than the three we need, and pick the top 80 articles for demonstration purposes.

In [6]:
# Ensure there is no duplicated text in the headers
def remove_duplicates(text):
    return re.sub(r'((\b\w+\b.{1,2}\w+\b)+).+\1', r'\1', text, flags=re.I)

df ['text'] = df['text'].apply(remove_duplicates)

# Keep only selected languages
languages = ['English', 'Spanish', 'Danish']
df = df.loc[df['lang'].isin(languages)]

# Pick the top 80 longest articles
df['text_length'] = df['text'].str.len()
df.sort_values(by=['text_length'], ascending=False, inplace=True)
top_80_df = df[:80]

# Language distribution
top_80_df['lang'].value_counts()

lang
Spanish    33
English    29
Danish     18
Name: count, dtype: int64

# Embed and index documents

Now, we want to embed our documents and store the embeddings. The embeddings are very large vectors that encapsulate the semantic meaning of our document. In particular, we use Cohere’s `embed-multilingual-v3.0` model, which creates embeddings with 1,024 dimensions.

With the v3.0 embeddings models, we need to specify the input_type parameter to indicate the nature of the document. In semantic search applications, this is either `search_document`, which is for the documents to search, or `search_query`, which is for the search query that we’ll define later.
We also keep track of the language and translation of the document to enrich the display of the results.

Next, we create a search index using the `hnsw` vector library. This stores the embeddings in an index, which makes searching the documents more efficient.

In [7]:
# Embed documents
docs = top_80_df['text'].to_list()
docs_lang = top_80_df['lang'].to_list()
translated_docs = top_80_df['translation'].to_list() #for reference when returning non-English results
doc_embs = co.embed(texts=docs,
                    model="cohere.embed-multilingual-v3",
                    input_type='search_document').embeddings

# Create a search index
index = hnswlib.Index(space='ip', dim=1024)
index.init_index(max_elements=len(doc_embs), ef_construction=512, M=64)
index.add_items(doc_embs, list(range(len(doc_embs))))

Next, we build a function that takes a query as input, embeds it, and finds the four headers more closely related to it.

In [8]:
# Retrieval of 4 closest docs to query
def retrieval(query):
    # Embed query and retrieve results
    query_emb = co.embed(texts=[query],
                         model="cohere.embed-multilingual-v3",
                         input_type="search_query").embeddings
    doc_ids = index.knn_query(query_emb, k=3)[0][0] # we will retrieve 4 closest neighbors
    
    # Print and append results
    print(f"QUERY: {query.upper()} \n")
    retrieved_docs, translated_retrieved_docs = [], []
    
    for doc_id in doc_ids:
        # Append results
        retrieved_docs.append(docs[doc_id])
        translated_retrieved_docs.append(translated_docs[doc_id])
    
        # Print results
        print(f"ORIGINAL ({docs_lang[doc_id]}): {docs[doc_id]}")
        if docs_lang[doc_id] != "English":
            print(f"TRANSLATION: {translated_docs[doc_id]} \n----")
        else:
            print("----")
    print("END OF RESULTS \n\n")
    return retrieved_docs, translated_retrieved_docs

Let’s now try to query the index with a couple of examples, one each in English and Danish.

As for results from the English query, notice how the retrieval system was able to surface documents similar in meaning, i.e., data science vs. AI. This is something that keyword-based search systems would not be able to capture.

As for results from the Danish query, it highlights the ability to perform cross-lingual search with the Embed multilingual model. You can enter a query in one language and get relevant search results that span other languages.
Another observation here is that the English acronym “PP&E” stands for “property, plant, and equipment,” and the model was able to connect it to the query.

In [9]:
queries = [
    "Can data science help meet sustainability goals?", # English example
    "Hvor kan jeg finde den seneste danske boligplan?" # Danish example - "Where can I find the latest Danish property plan?"
]

for query in queries:
    retrieval(query)

QUERY: CAN DATA SCIENCE HELP MEET SUSTAINABILITY GOALS? 

ORIGINAL (English): Using AI to better manage the environment could reduce greenhouse gas emissions, boost global GDP by up to 38m jobs by 2030
----
ORIGINAL (English): Quality of business reporting on the Sustainable Development Goals improves, but has a long way to go to meet and drive targets.
----
ORIGINAL (English): Only 10 years to achieve Sustainable Development Goals but businesses remain on starting blocks for integration and progress
----
END OF RESULTS 


QUERY: HVOR KAN JEG FINDE DEN SENESTE DANSKE BOLIGPLAN? 

ORIGINAL (Danish): Nyt fra CFOdirect: Ny PP&E-guide, FAQs om den nye leasingstandard, podcast om udfordringerne ved implementering af leasingstandarden og meget mere
TRANSLATION: New from CFOdirect: New PP&E guide, FAQs on the new leasing standard, podcast on the challenges of implementing the leasing standard and much more 
----
ORIGINAL (Danish): Lovforslag fremlagt om rentefri lån, udskudt frist for lønsums

Semantic search applications, enabled by text embeddings, offer a significantly more effective approach to retrieving and analyzing information. Cohere's Embed model can do this across over 100 languages. Its application in fields like financial analysis, as demonstrated in this chapter, shows how it can transform data retrieval and processing tasks, saving time and improving accuracy.